In [1]:
import numpy as np
import pandas as pd
import os
path = os.getcwd()

In [2]:
size = '4'
path = './All_spreadsheet_'+size

In [3]:
### read evenly-spaced discretiztion of empirical de-meaned CDF of div growth
input_1 = pd.read_excel(path+'/1.xlsx',header=None)
states_empirical_CDF = input_1.iloc[:-1,0].values

### read quadrature abscissa for theta_hat
states_theta_hat = input_1.iloc[:-1,1].values

### read quadrature abscissa for theta_i,i=1...1000
def states_from_simulations(i):
    # set i from 1...1000
    return input_1.iloc[:-1,i+1].values

### read mean for div growth
div_growth_mean = input_1.iloc[-1,0]

### read KL_i,i=1...1000
KL = input_1.iloc[-1,2:].values

### read stationary distribution for markov chain approx from theta_hat
input_2 = pd.read_excel(path+'/2.xlsx',header=None)
dist_theta_hat = input_2.iloc[:,0].values

### read stationary distribution for markov chain approx from theta_i,i=1...1000
def dist_from_simulations(i):
    # set i from 1...1000
    return input_2.iloc[:,i].values

### read transition matrix for markov chain approx from theta_hat
transition_matrix_theta_hat = pd.read_excel(path+'/3.xlsx',header=None).values

### read transition matrix for markov chain approx from theta_i,i=1...1000
def trans_matrix_from_simulations(i):
    # set i from 1...1000
    return pd.read_excel(path+'/'+str(i+3)+'.xlsx',header=None).values

In [7]:
dist_from_simulations(0)

array([0.25, 0.25, 0.25, 0.25])

In [4]:
states_theta_hat

array([-0.03548438, -0.01127826,  0.01127826,  0.03548438])

In [5]:
def states_to_dividend_growth(states : np.array, a = 0.01037):
    dividend_growth = np.zeros((len(states),1))
    for indx, element in np.ndenumerate(states):
        dividend_growth[indx] += np.exp(a+element)
    return dividend_growth

dividend_growth = states_to_dividend_growth(states_theta_hat)

In [6]:
def growth_to_H(dividend_growth, b=0.03630):
    b_matrix = np.array([[np.exp(b), np.exp(-b)],
                         [np.exp(b), np.exp(-b)]])
    one = np.ones((1,len(dividend_growth)))
    H = np.kron(np.matmul(dividend_growth,one),
                    b_matrix)
    return H

H = growth_to_H(dividend_growth)

In [7]:
def sdf_equation(state):
    return np.exp(-0.8970 + (1.2038*state))

def states_to_sdf(states : np.array, sdf_equation : callable):
    sdf = np.zeros((len(states),1))
    for indx, element in np.ndenumerate(states):
        sdf[indx] += sdf_equation(element)
    return sdf

sdf = states_to_sdf(states_theta_hat, sdf_equation)

In [8]:
def parameters_to_xi(gamma, function, b=0.03630):

    if function == 'CRRA':
        return -b*gamma
    if function == 'IES':
        return  0.0412 - (0.0775*gamma)
    else:
        raise Exception("Function must be either 'CRRA' or 'IES'")
        

gamma=2
function = 'CRRA'
xi = parameters_to_xi(gamma, function, b=0.03630)

In [9]:
def sdf_to_M(sdf: np.array, xi):
    xi_matrix = np.array([[np.exp(xi), np.exp(-xi)],
                         [np.exp(xi), np.exp(-xi)]])
    
    one = np.ones((1,len(dividend_growth)))
    M = np.kron(np.matmul(sdf,one),
                    xi_matrix)
    
    return M

M = sdf_to_M(sdf, xi)

In [10]:
M.shape

(8, 8)

In [11]:
def Psi_and_Pi_to_B(Psi, Pi):
    size = Psi.shape[0]
    B = np.zeros((size,size))
    for row in range(size):
        for pi, psi in zip(Pi[row], Psi[row]):
            B[row][row] += pi*psi
            
        B[row][row] /= size
    return B

Pi = np.kron(transition_matrix_theta_hat, 
            np.array([[1/2, 1/2],[1/2, 1/2]]))
Psi = np.multiply(H, M)
B = Psi_and_Pi_to_B(Psi, Pi)

In [12]:
def H_M_and_Pi_to_A(H, M, Pi):

    I = np.identity(H.shape[0])
    A = np.multiply(H, M)
    A = np.multiply(A, Pi)
    A = I - A
    return A

A = H_M_and_Pi_to_A(H, M, Pi)

In [13]:
C = np.matmul(np.linalg.inv(B),A)

In [14]:
unit = np.ones((C.shape[0],1))
unit /= C.shape[0]

v_128 = np.matmul(np.linalg.inv(C), unit)
v_128.sort()

In [15]:
d_half = states_empirical_CDF + div_growth_mean
d_half.shape

(4,)

In [16]:
def cdf_and_mean_to_d(states_empirical_CDF, div_growth_mean):

    d_half = states_empirical_CDF + div_growth_mean

    size = d_half.shape[0]
    d_full = np.zeros((2*size,1))

    for i, observation in enumerate(d_half):
        index = 2*i
        d_full[index] = observation
        if i == size-1:
            d_full[index+1] = observation
        else:
            ave = observation + d_half[i+1]
            ave /= 2
            d_full[index+1] = ave
            
    return d_full

d_full = cdf_and_mean_to_d(states_empirical_CDF, div_growth_mean)

In [17]:
d_full

array([[-0.28428393],
       [-0.19976881],
       [-0.1152537 ],
       [-0.03073858],
       [ 0.05377653],
       [ 0.13829165],
       [ 0.22280677],
       [ 0.22280677]])

In [ ]:
def C(gamma, utility_function):
    
    dividend_growth = states_to_dividend_growth(states_theta_hat)
    H = growth_to_H(dividend_growth)
    sdf = states_to_sdf(states_theta_hat, sdf_equation)
    xi = parameters_to_xi(gamma, utility_function, b=0.03630)
    M = sdf_to_M(sdf, xi)
    
    Pi = np.kron(transition_matrix_theta_hat, 
            np.array([[1/2, 1/2],[1/2, 1/2]]))
    Psi = np.multiply(H, M)
    B = Psi_and_Pi_to_B(Psi, Pi)
    
    A = H_M_and_Pi_to_A(H, M, Pi)
    
    C = np.matmul(np.linalg.inv(B),A)
    return C

In [30]:
folder = path+'/C_matrix/'

for utility_function in ['CRRA','IES']:
    for gamma in [2,5,10]:
        matrix = C(gamma, utility_function)
        df = pd.DataFrame(matrix)
        matrix_name = utility_function+'_'+str(gamma)
        df.to_excel(folder+matrix_name+'.xlsx',header=None,index=False)
        
        test_df = pd.read_excel(folder+matrix_name+'.xlsx', header=None)
        test = test_df.to_numpy()
        


In [ ]:
def model_growth(gamma, utility_function):
    
    dividend_growth = states_to_dividend_growth(states_theta_hat)
    H = growth_to_H(dividend_growth)
    sdf = states_to_sdf(states_theta_hat, sdf_equation)
    xi = parameters_to_xi(gamma, utility_function, b=0.03630)
    M = sdf_to_M(sdf, xi)
    
    Pi = np.kron(transition_matrix_theta_hat, 
            np.array([[1/2, 1/2],[1/2, 1/2]]))
    Psi = np.multiply(H, M)
    B = Psi_and_Pi_to_B(Psi, Pi)
    
    A = H_M_and_Pi_to_A(H, M, Pi)
    
    C = np.matmul(np.linalg.inv(B),A)
    
    unit = np.ones((C.shape[0],1))
    unit /= np.linalg.norm(unit)
    
    v = np.matmul(np.linalg.inv(C),
                      unit)
    
    d = cdf_and_mean_to_d(states_empirical_CDF, div_growth_mean)

    solution = d-v
    return np.matmul(solution.T, solution)

In [ ]:
for utility_function in ['CRRA','IES']:
    for gamma in [2,5,10]:
        growth = model_growth(gamma, utility_function)
        print('utility_function = ',utility_function,', gamma = ',gamma,', growth = ',growth)

utility_function =  CRRA , gamma =  2 , growth =  [[0.78690907]]
utility_function =  CRRA , gamma =  5 , growth =  [[0.78688321]]
utility_function =  CRRA , gamma =  10 , growth =  [[0.78676142]]
utility_function =  IES , gamma =  2 , growth =  [[0.78690301]]
utility_function =  IES , gamma =  5 , growth =  [[0.78677744]]
utility_function =  IES , gamma =  10 , growth =  [[0.78597848]]


In [ ]:
for utility_function in ['CRRA','IES']:
    for gamma in [2,5,10]:
        growth = model_growth(gamma, utility_function)
        print('utility_function = ',utility_function,', gamma = ',gamma,', growth = ',growth)

utility_function =  CRRA , gamma =  2 , growth =  [[0.78690907]]
utility_function =  CRRA , gamma =  5 , growth =  [[0.78688321]]
utility_function =  CRRA , gamma =  10 , growth =  [[0.78676142]]
utility_function =  IES , gamma =  2 , growth =  [[0.78690301]]
utility_function =  IES , gamma =  5 , growth =  [[0.78677744]]
utility_function =  IES , gamma =  10 , growth =  [[0.78597848]]


In [ ]:
def HHL_parameters(gamma, utility_function):
    dividend_growth = states_to_dividend_growth(states_theta_hat)
    H = growth_to_H(dividend_growth)
    sdf = states_to_sdf(states_theta_hat, sdf_equation)
    xi = parameters_to_xi(gamma, utility_function, b=0.03630)
    M = sdf_to_M(sdf, xi)
    
    Pi = np.kron(transition_matrix_theta_hat, 
            np.array([[1/2, 1/2],[1/2, 1/2]]))
    Psi = np.multiply(H, M)
    B = Psi_and_Pi_to_B(Psi, Pi)
    
    A = H_M_and_Pi_to_A(H, M, Pi)
    
    C = np.matmul(np.linalg.inv(B),A)
    
    unit = np.ones((C.shape[0],1))
    unit /= np.linalg.norm(unit)
    
    v = np.matmul(np.linalg.inv(C),
                      unit)

    return C, unit, v

In [ ]:
gamma = 2
utility_function = 'IES'
C, unit, v = HHL_parameters(gamma, utility_function)
#np.linalg.eigvals(C_128)

In [ ]:
v

In [ ]:
c_mat = np.asmatrix(C)
#c_mat /= np.linalg.norm(c_mat)
size = c_mat.shape[0]

hermitian = np.zeros((2*size,2*size))

for index, entry in np.ndenumerate(c_mat):
    hermitian[index[0],size+index[1]] = entry
    #print(hermitian[index[0],size+index[1]])

for index, entry in np.ndenumerate(c_mat.H):
    hermitian[size+index[0],index[1]] = entry
    
unit = np.ones((C.shape[0],1))
unit /= np.linalg.norm(unit)
b = np.kron([[1],[0]], unit)
x = np.kron([[0],[1]], v)

scale = 0.5/abs(max(np.linalg.eigvals(hermitian), key=abs))
#scale = np.linalg.norm(x)/np.linalg.norm(b)
#hermitian*=scale


In [ ]:
b

In [ ]:
unit = spla.expm(-2j*np.pi*hermitian)

In [ ]:
def project_eigenbasis(matrix, vector):
    projection = np.zeros(vector.shape, dtype=complex)
    eigen_values, eigen_vectors = np.linalg.eigh(matrix)
    #unitary = spla.expm(-1j * matrix * np.pi/max(eigen_vals))
    for i, e_vector in enumerate(eigen_vectors.T):
        #e_vec= np.reshape(e_vector, vector.shape)
        #print(np.linalg.norm(e_vec))
        amplitude = np.dot(e_vector, vector)
        #print(amplitude)
        projection[i] = amplitude
    return projection

In [ ]:
b_projection = project_eigenbasis(hermitian, b)
b_test = np.linalg.inv(np.linalg.eigh(hermitian)[1]).dot(b)
print(np.allclose(b_projection, b_test))
np.linalg.norm(b_projection)
b_projection

In [ ]:
def divide_eigen_vals(matrix, projection):
    new_projection = np.zeros(projection.shape, dtype=complex)
    eigen_vals = np.linalg.eigh(matrix)[0]
    for i, (proj, ev) in enumerate(zip(projection, eigen_vals)):
        #print(ev)
        new_projection[i] = proj/ev
    return new_projection

In [ ]:
def return_normalbasis(matrix, projection):
    normal = np.asmatrix(np.zeros(projection.shape, dtype=complex))
    print(projection.shape)
    eigen_vectors = np.linalg.eigh(matrix)[1].T
    for proj, vec in zip(projection, eigen_vectors):
        vec = vec.reshape(projection.shape)
        normal += complex(proj)*vec
    return normal

In [ ]:
x_projection = divide_eigen_vals(hermitian, b_projection)
x_test = np.linalg.inv(np.linalg.eigh(hermitian)[1]).dot(x)
np.linalg.norm(x_projection)

In [ ]:
x_solution = return_normalbasis(unit, x_projection)
x_norm = np.linalg.norm(x_solution)

In [ ]:
np.allclose(x, x_solution/x_norm)

In [83]:
eigen_vals = np.linalg.eigh(hermitian)[0]
eigen_vecs = np.linalg.eigh(hermitian)[1].T
print(np.linalg.norm(eigen_vecs[1]))
print(np.linalg.norm(hermitian.dot(eigen_vecs[1])))
print(eigen_vals[1])

0.9999999999999996
0.48889321998915647
-0.4888932199891565


In [43]:
x_inv_projection = project_eigenbasis(np.linalg.inv(unit), x)

In [97]:
#norm = np.sqrt(np.linalg.norm(x_projection))
x_projection = divide_eigen_vals(hermitian, b_projection)
x_test = np.linalg.inv(np.linalg.eigh(hermitian)[1]).dot(x)
np.linalg.norm(x_projection)

1.0000000000000004

In [101]:
x_scaled = x/np.linalg.norm(x)
np.allclose(x_scaled, x_solution)

True

In [ ]:
unit = spla.expm(-2j*np.pi*hermitian)

eigen_vals = np.linalg.eigh(unit)[0]
for i in range(len(x_test)):
    print(b_test[i]/x_test[i])
    print(eigen_vals[i])

In [ ]:
eigen_values, eigen_vectors = np.linalg.eigh(hermitian)
print(np.dot(eigen_vectors[:,0],b))
print(np.linalg.inv(eigen_vectors).dot(b)[0])


In [26]:
unit = ham.__array__()

NameError: name 'ham' is not defined

In [37]:
import scipy.linalg as spla
from qiskit.extensions import HamiltonianGate
from qiskit.quantum_info.operators.predicates import is_hermitian_matrix, is_unitary_matrix

In [28]:
spla.det(C_128, check_finite=True)

inf

In [29]:
test = np.matmul(hermitian, x)

In [30]:
scale = 1/abs(max(np.linalg.eigvals(hermitian), key=abs))

In [31]:
unit = spla.expm(-1j * hermitian * 2 * np.pi / scale)

In [32]:
def return_normalbasis(matrix, projection):
    normal = np.asmatrix(np.zeros(projection.shape, dtype=complex))
    print(projection.shape)
    eigen_vectors = np.linalg.eigh(matrix)[1].T
    for proj, vec in zip(projection, eigen_vectors):
        vec = vec.reshape(projection.shape)
        normal += complex(proj)*vec
    return normal

In [33]:
x_normal = return_normalbasis(hermitian, x_projection)

NameError: name 'x_projection' is not defined

In [ ]:
x_normal

In [ ]:
def project_standardbasis(matrix, projection):
    vector = np.zeros(projection.shape)
    eigen = np.linalg.eigh(matrix)
    e_vals = eigen[0]
    e_vectors = eigen[1]
    for i, e_vector in enumerate(e_vectors):
        e_vec= np.reshape(e_vector, vector.shape)
        vec = projection[i]*e_vec
        #print(np.linalg.norm(e_vec))
        
        #print(amplitude)
        vector += vec
    return projection

In [ ]:
hermitian_norm = hermitian*norm

In [ ]:
projection_b = project_eigenbasis(hermitian_norm, b)
np.linalg.norm(projection_b)

In [ ]:
project_standardbasis(hermitian_norm, projection_b)

In [ ]:
projection_x = project_eigenbasis(hermitian, x)
np.linalg.norm(projection_x)

In [ ]:
eigs = np.linalg.eigh(hermitian)[0]

In [34]:
for i in range(256):
    proj_eig = projection_b[i]/projection_x[i]
    print(eigs[i]/proj_eig)

NameError: name 'projection_b' is not defined

In [35]:
backward_test = np.matmul(np.linalg.inv(hermitian), b)
np.linalg.norm(backward_test)

6.580051807348764

In [36]:
projection_backward_test = project_eigenbasis(hermitian, backward_test)
np.linalg.norm(projection_backward_test)

NameError: name 'project_eigenbasis' is not defined

In [262]:
np.allclose(projection, projection_forward_test)

True

In [257]:
np.linalg.norm(forward_projection)

0.16008491002580405

In [255]:
for i, val in enumerate(np.linalg.eigh(hermitian)[0]):
    forward_projection = val*projection_inv[i]
    #print(projection[i]/projection_inv[i])
    #print(projection[i])
    #print(projection_change)
    #print(eigvals2[i])
    #print(val)

In [340]:
def classical_hhl(matrix, projection):
    result = np.zeros(projection.shape)
    e_vals = np.linalg.eigh(matrix)[0]
    e_vectors = np.linalg.eigh(matrix)[1]
    for i, e_vector in enumerate(e_vectors):
        #print(amplitude)
        #print(projection.shape)
        vector = np.reshape(e_vector, projection.shape)
        vector *= projection[i]
        #print(e_vals[i])
        vector /= e_vals[i]
        #print(vector.shape)
        result += vector
        
    return result

In [341]:
forward_test = np.matmul(hermitian, projection_b)

In [356]:
result = classical_hhl(hermitian_norm, projection_b)
res_norm = np.linalg.norm(result)

In [358]:
x

array([[0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.   

In [23]:
from qiskit.quantum_info import Statevector, ScalarOp

In [24]:
gamma = 5
utility_function = 'IES'

v, d = quantum_model_growth(gamma, utility_function)

In [25]:
v_sv = Statevector(v)
d_sv = Statevector(d)

In [26]:
d_op = d_sv.to_operator()
Act = ScalarOp(d_op.dim)

In [27]:
#v_sv.expectation_value(Act)

In [28]:
d_op.dim

(128, 128)

In [29]:
Act

ScalarOp((128, 128), coeff=1)